In [11]:
import pandas as pd
from fredapi import Fred
from config import key

In [12]:
fred = Fred(api_key= key)

In [13]:
pd.set_option('display.max_columns', 999)
pd.set_option('display.max_rows', 50)

In [14]:
fred.search('us ism').T

series id,USSLIND,CASLIND,FLSLIND,TXSLIND,MISLIND,NYSLIND,COSLIND,OKSLIND,ALSLIND,OHSLIND,WISLIND,TNSLIND,MNSLIND,WASLIND,AZSLIND,ORSLIND,INSLIND,NVSLIND,NCSLIND,NESLIND,PASLIND,ILSLIND,SCSLIND,LASLIND,IASLIND,KYSLIND,VASLIND,MASLIND,HISLIND,NJSLIND,KSSLIND,IDSLIND,MOSLIND,ARSLIND,GASLIND,CTSLIND,UTSLIND,RISLIND,AKSLIND,WVSLIND,MSSLIND,DESLIND,WYSLIND,NDSLIND,VTSLIND,MDSLIND,NHSLIND,NMSLIND,SDSLIND,MESLIND,MTSLIND
id,USSLIND,CASLIND,FLSLIND,TXSLIND,MISLIND,NYSLIND,COSLIND,OKSLIND,ALSLIND,OHSLIND,WISLIND,TNSLIND,MNSLIND,WASLIND,AZSLIND,ORSLIND,INSLIND,NVSLIND,NCSLIND,NESLIND,PASLIND,ILSLIND,SCSLIND,LASLIND,IASLIND,KYSLIND,VASLIND,MASLIND,HISLIND,NJSLIND,KSSLIND,IDSLIND,MOSLIND,ARSLIND,GASLIND,CTSLIND,UTSLIND,RISLIND,AKSLIND,WVSLIND,MSSLIND,DESLIND,WYSLIND,NDSLIND,VTSLIND,MDSLIND,NHSLIND,NMSLIND,SDSLIND,MESLIND,MTSLIND
realtime_start,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00
realtime_end,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00,2020-02-15 00:00:00
title,Leading Index for the United States,Leading Index for California,Leading Index for Florida,Leading Index for Texas,Leading Index for Michigan,Leading Index for New York,Leading Index for Colorado,Leading Index for Oklahoma,Leading Index for Alabama,Leading Index for Ohio,Leading Index for Wisconsin,Leading Index for Tennessee,Leading Index for Minnesota,Leading Index for Washington,Leading Index for Arizona,Leading Index for Oregon,Leading Index for Indiana,Leading Index for Nevada,Leading Index for North Carolina,Leading Index for Nebraska,Leading Index for Pennsylvania,Leading Index for Illinois,Leading Index for South Carolina,Leading Index for Louisiana,Leading Index for Iowa,Leading Index for Kentucky,Leading Index for Virginia,Leading Index for Massachusetts,Leading Index for Hawaii,Leading Index for New Jersey,Leading Index for Kansas,Leading Index for Idaho,Leading Index for Missouri,Leading Index for Arkansas,Leading Index for Georgia,Leading Index for Connecticut,Leading Index for Utah,Leading Index for Rhode Island,Leading Index for Alaska,Leading Index for West Virgini

In [18]:
pmi = pd.DataFrame(fred.get_series('USSLIND'))

In [19]:
pmi

,0
1982-01-01,-0.89
1982-02-01,-0.38
1982-03-01,-0.15
1982-04-01,-0.12
1982-05-01,-0.07
...,...
2019-08-01,1.30
2019-09-01,1.48
2019-10-01,1.34
2019-11-01,1.33
